# This notebook is for reference only
It relies on the Prolific ID of some participants, which has been redacted to ensure anonymity

In [1]:
import csv

import numpy as np
import pandas as pd
import os

from datetime import datetime
from glob import glob

pd.set_option('display.max_columns', 100)

In [2]:
SHOULD_SCRUB_ID = True

In [3]:
DATA_PATH = sorted(glob('../data/*.csv'))[-1] # retrieve latest dataset
OUTPUT_DIR = '../data/processed'

ID_BLACKLIST = [
    # REDACTED
]

ID_OVERCONFIDENT_PROGRAMMERS = [
    # REDACTED
]

ID_OVERCONFIDENT_WEB3 = [
    # REDACTED
]

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load
- Eliminate unnamed columns
- Remove non-prolific entries
- Remove full question text (use short question name only)

In [4]:
df = pd.read_csv(DATA_PATH, skiprows=[1])

# remove unnamed columns
cols = df.columns
df = df[[c for c in cols if not c.startswith("Unnamed:")]]

# remove non-prolific entries
df = df[df['id'].notnull()]
df = df[df['id'] != 'Doris']

# remove blacklisted rows
df = df[~df['id'].isin(ID_BLACKLIST)]

# CORRECTIONS
# guy who mistakenly says he owns usdt
df.loc[df['usdt_main_storage'] == "I don't own USDT", 'usdt_main_storage'] = np.nan

# remove old data on metamask efficiency(diff qn phrasing)
df.loc[df['mm_discover_fail_num'].isnull(), 'mm_discover_succeed_full_num'] = np.nan
df.loc[df['mm_discover_fail_num'].isnull(), 'mm_understand_succeed_full_num'] = np.nan

print(f"{len(df)} entries loaded.")

121 entries loaded.


# Add sources columns

In [5]:
from collections import Counter

# what sources are there?
srcs = Counter()
for l in df.sources.str.split(', '):
    if not isinstance(l, list):
        continue
    srcs = srcs + Counter(l)
        
srcs

Counter({'Medium article': 41,
         'The smart contract code': 25,
         'YouTube video': 89,
         'Information from other people': 64,
         'Google': 1,
         'Whitepaper/greypaper': 42,
         'Etherscan': 30,
         'Official documentation that is not the whitepaper/greypaper': 18,
         'Experience moving assets over the years': 1,
         'and many mistakes.': 1,
         'Security tools (e.g. MythX)': 9,
         'cold storage wallet transfers': 1,
         'Reddit': 1,
         'BSCSCAN': 1})

In [6]:
SRC_BLOCKCHAIN_EXPLORER = 'is_source_blockchain_explorer'
SRC_CODE = 'is_source_code'
SRC_MEDIUM = 'is_source_medium'
SRC_OFFICIAL_NONPAPER = 'is_source_official_nonpaper'
SRC_OFFICIAL_PAPER = 'is_source_official_paper'
SRC_OTHER_PEOPLE = 'is_source_other_people'
SRC_SECURITY_TOOL = 'is_source_security_tool'
SRC_YOUTUBE = 'is_source_youtube'

SRC_COLS_ATOMIC = [
    SRC_BLOCKCHAIN_EXPLORER,
    SRC_CODE,
    SRC_MEDIUM,
    SRC_OFFICIAL_NONPAPER,
    SRC_OFFICIAL_PAPER,
    SRC_OTHER_PEOPLE,
    SRC_SECURITY_TOOL,
    SRC_YOUTUBE
]

SRC_COLS_OFFICIAL = [SRC_CODE, SRC_OFFICIAL_NONPAPER, SRC_OFFICIAL_PAPER]

SRC_OFFICIAL = 'is_used_official_source'
SRC_UNOFFICIAL = 'is_used_unofficial_source'

SRC_COLS_AGG = [
    SRC_OFFICIAL,
    SRC_UNOFFICIAL
]

SRCS = {
    SRC_BLOCKCHAIN_EXPLORER: {'Etherscan', 'BSCSCAN'},
    SRC_CODE: 'The smart contract code',
    SRC_MEDIUM: 'Medium article',
    SRC_OFFICIAL_NONPAPER: 'Official documentation that is not the whitepaper/greypaper',
    SRC_OFFICIAL_PAPER: 'Whitepaper/greypaper',
    SRC_OTHER_PEOPLE: 'Information from other people',
    SRC_SECURITY_TOOL: 'Security tools (e.g. MythX)',
    SRC_YOUTUBE: 'YouTube video',
}

for src, matches in SRCS.items():
    if not isinstance(matches, set):
        matches = {matches}
        
    f = lambda l: any(m in l for m in matches) if isinstance(l, list) else  np.nan
    df[src] = df.sources.str.split(', ').apply(f)
    
    
# src official
df[SRC_OFFICIAL] = df[SRC_CODE] | df[SRC_OFFICIAL_PAPER] | df[SRC_OFFICIAL_NONPAPER]
df.loc[df['sources'].isnull(), SRC_OFFICIAL] = np.nan

# src unofficial
df[SRC_UNOFFICIAL] = df[SRC_BLOCKCHAIN_EXPLORER] | df[SRC_MEDIUM] | df[SRC_OTHER_PEOPLE] | df[SRC_SECURITY_TOOL] | df[SRC_YOUTUBE]
df.loc[df['sources'].isnull(), SRC_UNOFFICIAL] = np.nan

# Set values for expl_pref
| Original | New |
|---|---|
| Diagram Description type | diag |
| Natural Language Description Type | natlang |
| Both are equally good | indifferent_positive |
| Neither (both are equally bad) | indifferent_negative |

In [7]:
df['expl_pref'] = df['expl_pref'].replace({
    'Diagram Description type': 'diag',
    'Natural Language Description type': 'natlang',
    'Both are equally good': 'indifferent_positive',
    'Neither (both are equally bad)': 'indifferent_negative',
})

# Add Explanation Preference

In [8]:
df['is_expl_prefers_natlang'] = df['expl_pref'] == 'natlang'
df['is_expl_prefers_diag'] = df['expl_pref'] == 'diag'
df['is_expl_prefers_indifferent_positive'] = df['expl_pref'] == 'indifferent_positive'
df['is_expl_prefers_indifferent_negative'] = df['expl_pref'] == 'indifferent_negative'
df['is_expl_prefers_indifferent_negative'].sum()

7

# Add Proficiency Columns

In [9]:
IS_WEB3 = 'is_web3'
IS_NOT_WEB3 = 'is_not_web3'
df[IS_WEB3] = df['web3_proficiency_num'] > 1
df[IS_NOT_WEB3] = ~df[IS_WEB3]

IS_GOOD_WEB3 = 'is_good_web3'
IS_NOT_GOOD_WEB3 = 'is_not_good_web3'
df[IS_GOOD_WEB3] = (df['web3_proficiency_num'] > 3) & (~df['id'].isin(ID_OVERCONFIDENT_WEB3))
df[IS_NOT_GOOD_WEB3] = ~df[IS_GOOD_WEB3]

IS_PROGRAMMER = 'is_programmer'
IS_NOT_PROGRAMMER = 'is_not_programmer'
df[IS_PROGRAMMER] = df['programming_proficiency_num'] > 1
df[IS_NOT_PROGRAMMER] = ~df[IS_PROGRAMMER]

IS_GOOD_PROGRAMMER = 'is_good_programmer'
IS_NOT_GOOD_PROGRAMMER = 'is_not_good_programmer'
df[IS_GOOD_PROGRAMMER] = (df['programming_proficiency_num'] > 3) & (~df['id'].isin(ID_OVERCONFIDENT_PROGRAMMERS))
df[IS_NOT_GOOD_PROGRAMMER] = ~df[IS_GOOD_PROGRAMMER]

IS_GOOD_ANTICIPATE_BEHAVIOR = 'is_good_anticipate_behavior'
IS_NOT_GOOD_ANTICIPATE_BEHAVIOR = 'is_not_good_anticipate_behavior'
df[IS_GOOD_ANTICIPATE_BEHAVIOR] = df['anticipate_behavior_num'] > 3
df[IS_NOT_GOOD_ANTICIPATE_BEHAVIOR] = ~df[IS_GOOD_ANTICIPATE_BEHAVIOR]

IS_GOOD_TRUST_BEHAVIOR = 'is_good_trust_behavior'
IS_NOT_GOOD_TRUST_BEHAVIOR = 'is_not_good_trust_behavior'
df[IS_GOOD_TRUST_BEHAVIOR] = df['trust_behavior_num'] > 3
df[IS_NOT_GOOD_TRUST_BEHAVIOR] = ~df[IS_GOOD_TRUST_BEHAVIOR]

# Add USDT Main Storage Columns

In [10]:
MAIN_STORAGE_WALLET = 'is_usdt_main_storage_wallet'
MAIN_STORAGE_NONWALLET = 'is_usdt_main_storage_nonwallet'

df[MAIN_STORAGE_WALLET] = df['usdt_main_storage'].isin([
    'Ethereum Wallet (e.g. MetaMask)',
    'ETH wallet via Ledger NanoX',
    'cold storage hardware wallet'
])
df.loc[df['usdt_main_storage'].isnull(), MAIN_STORAGE_WALLET] = np.nan

df[MAIN_STORAGE_NONWALLET] = df['usdt_main_storage'].isin([
    'Exchange (e.g. Binance)',
    'Nexo',
    'C4W' # some kinda reward pool idk lmao
])
df.loc[df['usdt_main_storage'].isnull(), MAIN_STORAGE_NONWALLET] = np.nan

df.usdt_main_storage.value_counts()

usdt_main_storage
Exchange (e.g. Binance)                 31
Ethereum Wallet (e.g. MetaMask)         21
Nexo                                     1
ETH wallet via Ledger NanoX              1
C4W                                      1
cold storage hardware wallet             1
No longer own - but was in a wallet.     1
Name: count, dtype: int64

# Change `aware` → `unaware`

In [11]:
aware_cols = [c for c in df.columns if '_aware' in c]
for col in aware_cols:
    new_col_name = col.replace('aware', 'unaware')
    if col.endswith('num'):
        df[new_col_name] = 6 - df[col]
    else:
        df[new_col_name] = df[col]
    if 'consortium_reject' in col:
        print(col, new_col_name)
        display(df[new_col_name])
    del df[col]
    
df = df[sorted(df.columns)]

grj_consortium_reject_aware_num grj_consortium_reject_unaware_num


/var/folders/k8/zk3t1hw970s24hcwklx_31ww0000gn/T/ipykernel_75102/1295994774.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col_name] = df[col]
/var/folders/k8/zk3t1hw970s24hcwklx_31ww0000gn/T/ipykernel_75102/1295994774.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col_name] = 6 - df[col]
/var/folders/k8/zk3t1hw970s24hcwklx_31ww0000gn/T/ipykernel_75102/1295994774.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perfo

5      5.0
6      5.0
7      4.0
8      5.0
9      5.0
      ... 
126    5.0
127    3.0
128    5.0
129    3.0
130    4.0
Name: grj_consortium_reject_unaware_num, Length: 121, dtype: float64

grj_consortium_reject_aware_why grj_consortium_reject_unaware_why


/var/folders/k8/zk3t1hw970s24hcwklx_31ww0000gn/T/ipykernel_75102/1295994774.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col_name] = df[col]


...

5      5.0
6      4.0
7      5.0
8      5.0
9      2.0
      ... 
126    5.0
127    5.0
128    4.0
129    4.0
130    4.0
Name: prj_consortium_reject_unaware_num, Length: 121, dtype: float64

prj_consortium_reject_aware_why prj_consortium_reject_unaware_why


/var/folders/k8/zk3t1hw970s24hcwklx_31ww0000gn/T/ipykernel_75102/1295994774.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col_name] = df[col]


5      I dont even know what  "a consortium of Tether...
6                 They wouldn't known the reason of this
7      I didn't know that was possible to reject tran...
8      i wasn't aware voting could take place for tra...
9                                        prior in survey
                             ...                        
126                                     would not happen
127    I don't think meetings are collected with such...
128    This is surprising to me because they charged ...
129               I wasn't aware that that could happen.
130                     I had no idea this could happen.
Name: prj_consortium_reject_unaware_why, Length: 121, dtype: object

/var/folders/k8/zk3t1hw970s24hcwklx_31ww0000gn/T/ipykernel_75102/1295994774.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col_name] = 6 - df[col]
/var/folders/k8/zk3t1hw970s24hcwklx_31ww0000gn/T/ipykernel_75102/1295994774.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col_name] = df[col]
/var/folders/k8/zk3t1hw970s24hcwklx_31ww0000gn/T/ipykernel_75102/1295994774.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perfo

# Make binary columns into boolean

In [ ]:
COLS = [
    'is_usdt_owned',
    'is_usdt_transferred_from_wallet',
    'is_usdt_transferred_to_wallet',
    "is_read_code",
    "is_believe_tether_decentralized",
]

for col in COLS:
    df.loc[df[col] == 'Yes', col.replace('is_', 'is_not_')] = False
    df.loc[df[col] == 'Yes', col] = True
    df.loc[df[col] == 'No', col.replace('is_', 'is_not_')] = True
    df.loc[df[col] == 'No', col] = False
df[COLS]

# Export data

In [13]:
filename = datetime.now().strftime("%Y-%m-%d_%H%M.csv")
path = os.path.join(OUTPUT_DIR, filename)
data = df
if SHOULD_SCRUB_ID:
    data = data[[c for c in data.columns if c != 'id']]
    data = data[[c for c in data.columns if c != 'prolific_id']]
df.to_csv(path)
print(f'Saved to {path}')

Saved to ../data/processed/2023-08-02_1020.csv
